<a href="https://colab.research.google.com/github/javahedi/QuantumStateTomography-ML/blob/main/test_numpy_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
import os
import glob
import csv
import pytest

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path= "/content/drive/MyDrive/SL_data/"
cvs_names=glob.glob(f'{path}*.csv')
bank_names=glob.glob(f'{path}*bank.npy')
weight_names=glob.glob(f'{path}*weights.npy')


# Select a single file
ind=0
with open(f'{cvs_names[ind]}','r') as f:
    csvreader =csv.reader(f,delimiter=",")
    data_list=list(csvreader)


banks_list   = []
weights_list = []
for id, name in enumerate(cvs_names):
    bank    = np.load(bank_names[id],mmap_mode="r")
    weights = np.load(weight_names[id],mmap_mode="r")
    banks_list.append(bank)
    weights_list.append(weights)


#print(bank.shape)
#print(weights.shape)

#print(banks_list[-1].shape)
#print(weights_list[-1].shape)


banks_data    = np.stack(banks_list)
weights_data  = np.stack(weights_list)


print(banks_data.shape)
print(weights_data.shape)


(1000, 100, 100, 2, 2)
(1000, 100, 100)


In [4]:
class InfoGain:
    def information_gain(self, angles, bank_particles, weights):
        angle_dict = {
            "theta": angles[0],
            "phi": angles[1]
        }
        best_guess = np.array(np.einsum('i...,i', bank_particles, weights))
        return self.adaptive_cost_func(angle_dict, bank_particles, weights, best_guess, 1)

    def adaptive_cost_func(self, angles, rhoBank, weights, bestGuess, nQubits):
        meshState = self.angles_to_state_vector(angles, nQubits)
        out = np.einsum('ij,ik->ijk', meshState, meshState.conj())
        K = self.Shannon_entropy(np.einsum('ijk,kj->i', out, bestGuess))
        J = self.Shannon_entropy(np.einsum('ijk,lkj->il', out, rhoBank))
        return np.real(K - np.dot(J, weights))

    def Shannon_entropy(self, prob):
        return np.real(np.sum(-(prob * np.log2(prob)), axis=0))

    def angles_to_state_vector(self, angles, nQubits):
        if nQubits == 1:
            tempMesh = np.array([np.cos(angles["theta"] / 2), np.exp(1j * angles["phi"]) * np.sin(angles["theta"] / 2)])
            meshState = np.array([tempMesh, self.get_opposing_state(tempMesh)])
        else:
            tempMeshA = np.array([np.cos(angles["thetaA"] / 2), np.exp(1j * angles["phiA"]) * np.sin(angles["thetaA"] / 2)])
            tempMeshB = np.array([np.cos(angles["thetaB"] / 2), np.exp(1j * angles["phiB"]) * np.sin(angles["thetaB"] / 2)])
            meshA = np.array([tempMeshA, self.get_opposing_state(tempMeshA)])
            meshB = np.array([tempMeshB, self.get_opposing_state(tempMeshB)])
            meshState = np.array([
                np.kron(meshA[0], meshB[0]),
                np.kron(meshA[0], meshB[1]),
                np.kron(meshA[1], meshB[0]),
                np.kron(meshA[1], meshB[1])
            ])
        return meshState

    def get_opposing_state(self, meshState):
        if meshState[1] == 0:
            return np.array([0, 1], dtype=complex)

        a = 1
        b = -np.conjugate(meshState[0]) / np.conjugate(meshState[1])
        norm = np.sqrt(a * np.conjugate(a) + b * np.conjugate(b))
        oppositeMeshState = np.array([a / norm, b / norm])
        return oppositeMeshState

info_gain = InfoGain()

In [5]:
NN_angle_pred=np.array([1,0.75]) # Format as [ theta, phi ]

# Select which step in ABME to predict
prediction_index=20

true_angle=np.array([float(data_list[prediction_index][3]),float(data_list[prediction_index][4])])

In [6]:
NN_angle_pred

array([1.  , 0.75])

In [7]:
true_angle

array([1.2499491, 1.7226485])

In [8]:

true_info_gain    = info_gain.information_gain(true_angle,bank[prediction_index],weights[prediction_index])
NN_pred_info_gain = info_gain.information_gain(NN_angle_pred,bank[prediction_index],weights[prediction_index])
print(f'Info gain difference: {true_info_gain - NN_pred_info_gain}')


Info gain difference: 0.00018063078880342642


In [9]:
class InfoGain_np_vectorized:
    def information_gain(self, angles, bank_particles, weights):
        angle_dict = {
            "theta": angles[0],
            "phi": angles[1]
        }
        best_guess = np.array(np.einsum('i...,i', bank_particles, weights))
        return self.adaptive_cost_func(angle_dict, bank_particles, weights, best_guess, 1)



    def adaptive_cost_func(self, angles, rhoBank, weights, bestGuess, nQubits):
        meshState = self.angles_to_state_vector(angles, nQubits)
        out = np.einsum('ij,ik->ijk', meshState, meshState.conj())
        K = self.Shannon_entropy(np.einsum('ijk,kj->i', out, bestGuess))
        J = self.Shannon_entropy(np.einsum('ijk,lkj->il', out, rhoBank))
        return np.real(K - np.dot(J, weights))

    def Shannon_entropy(self, prob):
        return np.real(np.sum(-(prob * np.log2(prob)), axis=0))

    def angles_to_state_vector(self, angles, nQubits):
        if nQubits == 1:
            tempMesh = np.array([np.cos(angles["theta"] / 2), np.exp(1j * angles["phi"]) * np.sin(angles["theta"] / 2)])
            meshState = np.array([tempMesh, self.get_opposing_state(tempMesh)])
        else:
            tempMeshA = np.array([np.cos(angles["thetaA"] / 2), np.exp(1j * angles["phiA"]) * np.sin(angles["thetaA"] / 2)])
            tempMeshB = np.array([np.cos(angles["thetaB"] / 2), np.exp(1j * angles["phiB"]) * np.sin(angles["thetaB"] / 2)])
            meshA = np.array([tempMeshA, self.get_opposing_state(tempMeshA)])
            meshB = np.array([tempMeshB, self.get_opposing_state(tempMeshB)])
            meshState = np.array([
                np.kron(meshA[0], meshB[0]),
                np.kron(meshA[0], meshB[1]),
                np.kron(meshA[1], meshB[0]),
                np.kron(meshA[1], meshB[1])
            ], dtype=complex)
        return meshState

    def get_opposing_state(self, meshState):
        if meshState[1].all() == 0:
            return np.array([0, 1], dtype=complex)

        a = 1
        b = -np.conjugate(meshState[0]) / np.conjugate(meshState[1])
        norm = np.sqrt(a * np.conjugate(a) + b * np.conjugate(b))
        oppositeMeshState = np.array([a / norm, b / norm], dtype=complex)
        return oppositeMeshState


info_gain_vectorized = InfoGain_np_vectorized()


In [10]:
y_predict_values     = np.stack([np.array([1,0.75]),np.array([1,0.0]),np.array([1,0.5])])
priction_indet=[20,40,50]
true_angle20=np.array([float(data_list[priction_indet[0]][3]),float(data_list[priction_indet[0]][4])])
true_angle40=np.array([float(data_list[priction_indet[1]][3]),float(data_list[priction_indet[1]][4])])
true_angle50=np.array([float(data_list[priction_indet[2]][3]),float(data_list[priction_indet[2]][4])])
y_true_values        = np.stack([true_angle20,true_angle40,true_angle50])
bank_particles_value = np.stack([bank[priction_indet[0]],bank[priction_indet[1]],bank[priction_indet[2]]])
weights_value        = np.stack([weights[priction_indet[0]],weights[priction_indet[2]],weights[priction_indet[2]]])

print(y_true_values.shape)
print(y_predict_values.shape)
print(bank_particles_value.shape)
print(weights_value.shape)

(3, 2)
(3, 2)
(3, 100, 2, 2)
(3, 100)


In [11]:
y_predict_values

array([[1.  , 0.75],
       [1.  , 0.  ],
       [1.  , 0.5 ]])

In [12]:
y_true_values

array([[ 1.2499491 ,  1.7226485 ],
       [ 1.47249126, -0.38898832],
       [ 0.52779046, -0.70772663]])

In [13]:

true_info_gains = np.array([
        info_gain_vectorized.information_gain(y_true_val, bank_particles_val, weights_val)
        for y_true_val, bank_particles_val, weights_val in zip(y_true_values, bank_particles_value, weights_value)
    ])
predict_info_gains = np.array([
        info_gain_vectorized.information_gain(y_pred_val, bank_particles_val, weights_val)
        for y_pred_val, bank_particles_val, weights_val in zip(y_predict_values, bank_particles_value, weights_value)
    ])

In [14]:
true_info_gains.sum()/len(true_info_gains)

0.0007745458218840207

In [15]:
for y1,y2 in zip(predict_info_gains,true_info_gains):
    print(f' {y1},     {y2},  and difference: {y2-y1}')

 0.0013689172477217015,     0.001549548036525128,  and difference: 0.00018063078880342642
 0.0003878619616848322,     0.00039571630569545935,  and difference: 7.854344010627123e-06
 0.0003923231874669031,     0.00037837312343147467,  and difference: -1.3950064035428422e-05


In [16]:
# Create a sample input with updated dimensions
angles_np         = np.random.rand(32, 100, 2)
true_angle        = angles_np + 0.01 * np.random.rand(32, 100, 2)
bank_particles_np = np.random.rand(32, 100, 50, 2, 2)
weights_np        = np.random.rand(32, 100, 50)
print(angles_np.shape)
print(bank_particles_np.shape)
print(weights_np.shape)

(32, 100, 2)
(32, 100, 50, 2, 2)
(32, 100, 50)


In [17]:
def loss_function(y_true, y_pred, banks, weights, lambda_weight=0.9):

    info_gain = InfoGain_np_vectorized()

    true_info_gains    = 0
    predict_info_gains = 0
    for i in range(y_true.shape[0]):
        true_info_gains += np.array([
                info_gain.information_gain(y_true_val, bank, weight)
                for y_true_val, bank, weight in zip(y_true[i,...], banks[i,...], weights[i,...])
            ]).sum()/y_true.shape[1]
        predict_info_gains += np.array([
                info_gain.information_gain(y_pred_val, bank, weight)
                for y_pred_val, bank, weight in zip(y_pred[i,...], banks[i,...], weights[i,...])
            ]).sum()/y_pred.shape[1]

    true_info_gains    /=y_true.shape[0]
    predict_info_gains /=y_true.shape[0]

    loss_infoGain = 1.0 / predict_info_gains

    return loss_infoGain


In [18]:
for _ in range(10):
    angles_np         = np.random.rand(32, 100, 2)
    true_angle        = angles_np + 0.01 * np.random.rand(32, 100, 2)
    bank_particles_np = np.random.rand(32, 100, 50, 2, 2)
    weights_np        = np.random.rand(32, 100, 50)
    output = loss_function(true_angle, angles_np, bank_particles_np, weights_np)

    print(output)


-0.009099908990904038
-0.0091297605405524
-0.009098407012855473
-0.009116362431607893
-0.009112758649579179
-0.009127038286505407
-0.009130459356104688
-0.00909509417105675
-0.009132914888961603
-0.009109659427015006


In [19]:

path         = "/content/drive/MyDrive/SL_data/"
cvs_names    = glob.glob(f'{path}*.csv')
bank_names   = glob.glob(f'{path}*bank.npy')
weight_names = glob.glob(f'{path}*weights.npy')

angles_list  = []
banks_list   = []
weights_list = []
for id, name in enumerate(cvs_names):
    data    = np.loadtxt( os.path.join(path,cvs_names[id]),delimiter=',',skiprows=1)
    bank    = np.load(bank_names[id],mmap_mode="r")
    weights = np.load(weight_names[id],mmap_mode="r")

    angles_list.append(data)
    banks_list.append(bank[1:])
    weights_list.append(weights[1:])

angles_data   = np.stack(angles_list)
banks_data    = np.stack(banks_list)
weights_data  = np.stack(weights_list)

print(angles_data.shape)
print(banks_data.shape)
print(weights_data.shape)


(1000, 99, 5)
(1000, 99, 100, 2, 2)
(1000, 99, 100)


In [20]:
# Set the percentage of data to use for validation
validation_split = 0.2

# Determine the number of samples to use for validation
num_validation_samples = int(angles_data.shape[0] * validation_split)

# Split the data into training and validation sets
train_data_angles  = angles_data[:-num_validation_samples]
train_data_banks   = banks_data[:-num_validation_samples]
train_data_weights = weights_data[:-num_validation_samples]

validation_data_angles  = angles_data[-num_validation_samples:]
validation_data_banks   = banks_data[-num_validation_samples:]
validation_data_weights = weights_data[-num_validation_samples:]

In [21]:
# Set batch size
batch_size = 16

# TensorFlow datasets API
train_dataset      = tf.data.Dataset.from_tensor_slices((train_data_angles, train_data_banks, train_data_weights)).batch(batch_size)
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_data_angles, validation_data_banks, validation_data_weights)).batch(batch_size)


In [22]:
# Check dimensions of the batches in training dataset
for batch_angles, batch_banks, batch_weights in train_dataset.take(1):
    print("Training Batch Shapes:")
    print("Angles Batch Shape:" , batch_angles.shape)
    print("Banks Batch Shape:"  , batch_banks.shape)
    print("Weights Batch Shape:", batch_weights.shape)

# Check dimensions of the batches in validation dataset
for batch_angles, batch_banks, batch_weights in validation_dataset.take(1):
    print("\nValidation Batch Shapes:")
    print("Angles Batch Shape:" , batch_angles.shape)
    print("Banks Batch Shape:"  , batch_banks.shape)
    print("Weights Batch Shape:", batch_weights.shape)

Training Batch Shapes:
Angles Batch Shape: (16, 99, 5)
Banks Batch Shape: (16, 99, 100, 2, 2)
Weights Batch Shape: (16, 99, 100)

Validation Batch Shapes:
Angles Batch Shape: (16, 99, 5)
Banks Batch Shape: (16, 99, 100, 2, 2)
Weights Batch Shape: (16, 99, 100)


In [23]:
class InfoGainNew:
    def information_gain(self, angles,bank_particles,weights):
        """
        Takes in angles as [theta,phi] and bank_particles and weigt as how they are given by the file.
        """
        angle_dict={
            "theta":angles[0],
            "phi": angles[1]
        }
        best_guess=np.array(np.einsum('i...,i',bank_particles,weights))
        return self.adaptive_cost_func(angle_dict,bank_particles,weights,best_guess)


    def adaptive_cost_func(self, angles,rhoBank,weights,bestGuess):
        """
        Computes the expected entropy reduction of the posterior (likelihood) distribution.
        The angles are taken in as a dictionary and indicate what mesaurement is to be perfomred.
        Noise correction is currently removed.
        """
        povm=self.angles_to_single_qubit_POVM(angles)
        # Computes the entropy of prior and posterior distributions. See 10.1103/PhysRevA.85.052120 for more details.
        K=self.Shannon_entropy(np.einsum('ijk,kj->i',povm,bestGuess))
        J=self.Shannon_entropy(np.einsum('ijk,lkj->il',povm,rhoBank))
        # Returns the negative values such that it becomes a minimization problem rather than maximization problem.
        return np.real(K-np.dot(J,weights))


    def Shannon_entropy(self, prob):
        """
        Returns the shannon entorpy of the probability histogram.
        """
        return np.real(np.sum(-(prob*np.log2(prob)),axis=0))

    def angles_to_single_qubit_POVM(self, angles):
        """
        Takes in measurement angles as dictionaries and returns the spin POVM as 2x2x2 complex array .
        For single qubit only.
        """
        up_state_vector=np.array([np.cos(angles["theta"]/2),np.exp(1j*angles["phi"])*np.sin(angles["theta"]/2)],dtype=complex)
        up_POVM=np.einsum("i,j->ij",up_state_vector,up_state_vector.conj())
        return np.array([up_POVM,np.eye(2)-up_POVM],dtype=complex)



info_gain_new = InfoGainNew()

In [26]:
y_predict_values     = np.stack([np.array([1,0.75]),np.array([1,0.0]),np.array([1,0.5])])
priction_indet=[20,40,50]
true_angle20=np.array([float(data_list[priction_indet[0]][3]),float(data_list[priction_indet[0]][4])])
true_angle40=np.array([float(data_list[priction_indet[1]][3]),float(data_list[priction_indet[1]][4])])
true_angle50=np.array([float(data_list[priction_indet[2]][3]),float(data_list[priction_indet[2]][4])])
y_true_values        = np.stack([true_angle20,true_angle40,true_angle50])
bank_particles_value = np.stack([bank[priction_indet[0]],bank[priction_indet[1]],bank[priction_indet[2]]])
weights_value        = np.stack([weights[priction_indet[0]],weights[priction_indet[2]],weights[priction_indet[2]]])

print(y_true_values.shape)
print(y_predict_values.shape)
print(bank_particles_value.shape)
print(weights_value.shape)

(3, 2)
(3, 2)
(3, 100, 2, 2)
(3, 100)


In [27]:
true_info_gains = np.array([
        info_gain_new.information_gain(y_true_val, bank_particles_val, weights_val)
        for y_true_val, bank_particles_val, weights_val in zip(y_true_values, bank_particles_value, weights_value)
    ])
predict_info_gains = np.array([
        info_gain_new.information_gain(y_pred_val, bank_particles_val, weights_val)
        for y_pred_val, bank_particles_val, weights_val in zip(y_predict_values, bank_particles_value, weights_value)
    ])

In [28]:
for y1,y2 in zip(predict_info_gains,true_info_gains):
    print(f' {y1},     {y2},  and difference: {y2-y1}')

 0.001368917247721757,     0.001549548036525128,  and difference: 0.0001806307888033709
 0.00038786196168494325,     0.0003957163056953483,  and difference: 7.854344010405079e-06
 0.0003923231874668476,     0.00037837312343147467,  and difference: -1.3950064035372911e-05


In [30]:
true_info_gains.shape

(3,)

In [113]:
class InfoGainTF:

    def __init__(self, debug=False):
        self.debug = debug

    @tf.function
    def information_gain(self, angles, bank_particles, weights):
        best_guess = tf.einsum('i...,i', bank_particles, weights)
        return self.adaptive_cost_func(angles, bank_particles, weights, best_guess)

    @tf.function
    def adaptive_cost_func(self, angles, rho_bank, weights, best_guess):
        povm = self.angles_to_single_qubit_POVM(angles)

        # Cast tensors to complex64 to match the data type of povm
        best_guess = tf.cast(best_guess, dtype=tf.complex64)
        rho_bank   = tf.cast(rho_bank, dtype=tf.complex64)

        if self.debug:
            # Check for NaN values
            self.check_numerics(best_guess, 'NaN values found in best_guess')
            self.check_numerics(rho_bank, 'NaN values found in rho_bank')

        K = self.shannon_entropy(tf.einsum('ijk,kj->i', povm, best_guess))
        J = self.shannon_entropy(tf.einsum('ijk,lkj->il', povm, rho_bank))

        if self.debug:
            # Check for NaN values
            self.check_numerics(K, 'NaN values found in K')
            self.check_numerics(J, 'NaN values found in J')

        return tf.math.real(K - tf.reduce_sum(J * weights))

    @staticmethod
    def check_numerics(tensor, message):
        real_part = tf.math.real(tensor)
        imag_part = tf.math.imag(tensor)
        tf.debugging.check_numerics(real_part, message=message + ' (Real part)')
        tf.debugging.check_numerics(imag_part, message=message + ' (Imaginary part)')

    @tf.function
    def shannon_entropy(self, prob):
        prob_float32 = tf.cast(prob, dtype=tf.float32)
        if self.debug:
            # Check for NaN values
            self.check_numerics(prob_float32, 'NaN values found in prob_float32')
        return tf.reduce_sum(-prob_float32 * tf.math.log(prob_float32 + 1e-10) / tf.math.log(2.0), axis=0)

    @tf.function
    def angles_to_single_qubit_POVM(self, angles):
        cos_component = tf.math.cos(angles[0] / 2)
        sin_component = tf.math.sin(angles[0] / 2)

        complex_part = tf.complex(tf.constant(0.0), angles[1])
        up_state_vector = tf.stack([tf.complex(cos_component, 0.0), tf.exp(complex_part) * tf.complex(sin_component, 0.0)], axis=0)

        up_POVM = tf.einsum("i,j->ij", up_state_vector, tf.math.conj(up_state_vector))

        # Cast tf.eye(2) and up_POVM to complex64 dtype
        eye_complex = tf.cast(tf.eye(2), dtype=tf.complex64)
        up_POVM = tf.cast(up_POVM, dtype=tf.complex64)

        return tf.stack([up_POVM, eye_complex - up_POVM], axis=0)




In [114]:
# Example usage
info_gain_tf = InfoGainTF(debug=True)
angles = tf.constant([0.1, 0.2], dtype=tf.float32)
bank_particles = tf.random.normal((10, 2, 2), dtype=tf.float32)
weights = tf.random.normal(shape=(10,), dtype=tf.float32)

result = info_gain_tf.information_gain(angles, bank_particles, weights)
print(result)

InvalidArgumentError: ignored

In [71]:


def test_einsum_operation():
    # Generate random data for testing
    bank_particles_data = np.random.rand(5, 2, 2).astype(np.float32)
    weights_data        = np.random.rand(5).astype(np.float32)

    # Convert NumPy arrays to TensorFlow tensors
    bank_particles_tf = tf.constant(bank_particles_data)
    weights_tf = tf.constant(weights_data)

    # Perform einsum operation in TensorFlow
    best_guess_tf = tf.einsum('i...,i', bank_particles_tf, weights_tf)

    # Perform the same einsum operation in NumPy for comparison
    best_guess_np = np.einsum('i...,i', bank_particles_data, weights_data)

    # Check if the TensorFlow and NumPy results are close
    assert np.allclose(best_guess_tf.numpy(), best_guess_np, rtol=1e-5), "Test Failed"




In [42]:
# Run the test
test_einsum_operation()

In [82]:


def angles_to_single_qubit_POVM_np(angles):
    up_state_vector = np.array(
        [np.cos(angles[0] / 2), np.exp(1j * angles[1]) * np.sin(angles[0] / 2)],
        dtype=complex
    )

    up_POVM = np.einsum("i,j->ij", up_state_vector, up_state_vector.conj())
    return np.array([up_POVM, np.eye(2) - up_POVM], dtype=complex)

def angles_to_single_qubit_POVM_tf(angles):
    cos_component = tf.math.cos(angles[0] / 2)
    sin_component = tf.math.sin(angles[0] / 2)

    complex_part = tf.complex(tf.constant(0.0), angles[1])
    up_state_vector = tf.stack([tf.complex(cos_component, 0.0), tf.exp(complex_part) * tf.complex(sin_component, 0.0)], axis=0)

    up_POVM = tf.einsum("i,j->ij", up_state_vector, tf.math.conj(up_state_vector))

    # Cast tf.eye(2) and up_POVM to complex64 dtype
    eye_complex = tf.cast(tf.eye(2), dtype=tf.complex64)
    up_POVM = tf.cast(up_POVM, dtype=tf.complex64)

    return tf.stack([up_POVM, eye_complex - up_POVM], axis=0)

def test_POVM():
    angles = np.random.rand(2).astype(np.float32)
    angles_tf = tf.constant(angles)

    # Perform calculations using NumPy
    POVM_np = angles_to_single_qubit_POVM_np(angles)

    # Perform calculations using TensorFlow
    POVM_tf = angles_to_single_qubit_POVM_tf(angles_tf)

    # Check if the TensorFlow and NumPy results are close
    assert np.allclose(POVM_tf.numpy(), POVM_np, rtol=1e-5), "Test Failed"
    #tf.debugging.assert_all_close(POVM_np, POVM_tf.numpy(), rtol=1e-5)




In [83]:
# Run the test
test_POVM()

To wrap a Python function and use it as a TensorFlow operation using ``tf.numpy_function()``, you need to create a TensorFlow-compatible function that calls your Python function. Here's a general example:

In [151]:


class InfoGainNew(tf.Module):
    def __init__(self):
        super(InfoGainNew, self).__init__()

    def information_gain(self, angles, bank_particles, weights):
        result = tf.numpy_function(self.information_gain_py, [angles, bank_particles, weights], tf.float64)
        return tf.convert_to_tensor(result, dtype=tf.float64)

    def information_gain_py(self, angles, bank_particles, weights):
        # Your existing Python code for information_gain
        angle_dict = {"theta": angles[0], "phi": angles[1]}
        best_guess = np.array(np.einsum('i...,i', bank_particles, weights))
        return self.adaptive_cost_func(angle_dict, bank_particles, weights, best_guess)

    def adaptive_cost_func(self, angles,rhoBank,weights,bestGuess):
        """
        Computes the expected entropy reduction of the posterior (likelihood) distribution.
        The angles are taken in as a dictionary and indicate what mesaurement is to be perfomred.
        Noise correction is currently removed.
        """
        povm=self.angles_to_single_qubit_POVM(angles)
        # Computes the entropy of prior and posterior distributions. See 10.1103/PhysRevA.85.052120 for more details.
        K=self.Shannon_entropy(np.einsum('ijk,kj->i',povm,bestGuess))
        J=self.Shannon_entropy(np.einsum('ijk,lkj->il',povm,rhoBank))
        # Returns the negative values such that it becomes a minimization problem rather than maximization problem.
        return np.real(K-np.dot(J,weights))


    def Shannon_entropy(self, prob):
        """
        Returns the shannon entorpy of the probability histogram.
        """
        return np.real(np.sum(-(prob*np.log2(prob)),axis=0))

    def angles_to_single_qubit_POVM(self, angles):
        """
        Takes in measurement angles as dictionaries and returns the spin POVM as 2x2x2 complex array .
        For single qubit only.
        """
        up_state_vector=np.array([np.cos(angles["theta"]/2),np.exp(1j*angles["phi"])*np.sin(angles["theta"]/2)],dtype=complex)
        up_POVM=np.einsum("i,j->ij",up_state_vector,up_state_vector.conj())
        return np.array([up_POVM,np.eye(2)-up_POVM],dtype=complex)



In [152]:
# Create an instance of the InfoGainNew class
info_gain_new = InfoGainNew()

# Example usage
angles_tf = tf.constant([0.1, 0.2], dtype=tf.float64)
bank_particles_tf = tf.constant(np.random.rand(10, 2, 2) + 1j * np.random.rand(10, 2, 2), dtype=tf.complex128)
weights_tf = tf.constant(np.random.rand(10), dtype=tf.float64)

result_tf = info_gain_new.information_gain(angles_tf, bank_particles_tf, weights_tf)

# Print the result
print(result_tf.numpy())

-12.849417911646256


In [153]:



# Example usage
angles_tf         = tf.constant(y_predict_values, dtype=tf.float64)
bank_particles_tf = tf.constant(bank_particles_value,  dtype=tf.complex128)
weights_tf        = tf.constant(weights_value, dtype=tf.float64)


print(angles_tf.shape)
print(bank_particles_tf.shape)
print(weights_tf.shape)



(3, 2)
(3, 100, 2, 2)
(3, 100)


In [154]:

result_tf = info_gain_new.information_gain(angles_tf[0,...], bank_particles_tf[0,...], weights_tf[0,...])

# Print the result
print(result_tf.numpy())

0.001368917247721757
